<a href="https://colab.research.google.com/github/Nagmashaik123/Nagma_INFO5731_Spring2021/blob/main/In_class_exercise_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The fifth In-class-exercise (2/23/2021, 20 points in total)

In exercise-03, I asked you to collected 500 textual data based on your own information needs (If you didn't collect the textual data, you should recollect for this exercise). Now we need to think about how to represent the textual data for text classification. In this exercise, you are required to select 10 types of features (10 types of features but absolutely more than 10 features) in the followings feature list, then represent the 500 texts with these features. The output should be in the following format:
![image.png](attachment:image.png)

The feature list:

* (1) tf-idf features
* (2) POS-tag features: number of adjective, adverb, auxiliary, punctuation, complementizer, coordinating conjunction, subordinating conjunction, determiner, interjection, noun, possessor, preposition, pronoun, quantifier, verb, and other. (select some of them if you use pos-tag features)
* (3) Linguistic features:
  * number of right-branching nodes across all constituent types
  * number of right-branching nodes for NPs only
  * number of left-branching nodes across all constituent types
  * number of left-branching nodes for NPs only
  * number of premodifiers across all constituent types
  * number of premodifiers within NPs only
  * number of postmodifiers across all constituent types
  * number of postmodifiers within NPs only
  * branching index across all constituent types, i.e. the number of right-branching nodes minus number of left-branching nodes
  * branching index for NPs only
  * branching weight index: number of tokens covered by right-branching nodes minus number of tokens covered by left-branching nodes across all categories
  * branching weight index for NPs only 
  * modification index, i.e. the number of premodifiers minus the number of postmodifiers across all categories
  * modification index for NPs only
  * modification weight index: length in tokens of all premodifiers minus length in tokens of all postmodifiers across all categories
  * modification weight index for NPs only
  * coordination balance, i.e. the maximal length difference in coordinated constituents
  
  * density (density can be calculated using the ratio of folowing function words to content words) of determiners/quantifiers
  * density of pronouns
  * density of prepositions
  * density of punctuation marks, specifically commas and semicolons
  * density of auxiliary verbs
  * density of conjunctions
  * density of different pronoun types: Wh, 1st, 2nd, and 3rd person pronouns
  
  * maximal and average NP length
  * maximal and average AJP length
  * maximal and average PP length
  * maximal and average AVP length
  * sentence length

* Other features in your mind (ie., pre-defined patterns)

In [126]:

from bs4 import BeautifulSoup
import requests
import re

#Web scrapping url selected is about IMDB Top ratings
url = 'http://www.imdb.com/chart/top'

# Sending request Url which returns python object response
response = requests.get(url)

#Parsing the page
soup = BeautifulSoup(response.text, 'lxml')

movieNames = soup.select('td.titleColumn')
links = [a.attrs.get('href') for a in soup.select('td.titleColumn a')]
crew = [a.attrs.get('title') for a in soup.select('td.titleColumn a')]
ratings = [b.attrs.get('data-value') for b in soup.select('td.posterColumn span[name=ir]')]
votes = [b.attrs.get('data-value') for b in soup.select('td.ratingColumn strong')]


#Step5: Save the text collected(Storing the data in the required format)
#I am converting to dictionary for better understanding

imdb = []
# Store each item into dictionary (data), then put those into a list (imdb)
for index in range(0, len(movieNames)):
    # Seperate movie into: 'place', 'title', 'year'
    movies = movieNames[index].get_text()
    movie = (' '.join(movies.split()).replace('.', ''))
    movie_title = movie[len(str(index))+1:-7]
    year = re.search('\((.*?)\)', movies).group(1)
    place = movie[:len(str(index))-(len(movie))]
    data = {"movie": movie_title,
            "year": year,
            "place": place,
            "crew": crew[index],
            "rating": ratings[index],            
            "link": links[index]}
    imdb.append(data)
df = pd.DataFrame(list(imdb), columns =['movie','year','place','crew','rating','link'])
df


,movie,year,place,crew,rating,link
0,The Shawshank Redemption,1994,1,"Frank Darabont (dir.), Tim Robbins, Morgan Fre...",9.223589219225481,/title/tt0111161/
1,The Godfather,1972,2,"Francis Ford Coppola (dir.), Marlon Brando, Al...",9.147988016587115,/title/tt0068646/
2,The Godfather: Part II,1974,3,"Francis Ford Coppola (dir.), Al Pacino, Robert...",8.980259662834687,/title/tt0071562/
3,The Dark Knight,2008,4,"Christopher Nolan (dir.), Christian Bale, Heat...",8.974749960095577,/title/tt0468569/
4,12 Angry Men,1957,5,"Sidney Lumet (dir.), Henry Fonda, Lee J. Cobb",8.934368439017161,/title/tt0050083/
...,...,...,...,...,...,...
245,The Invisible Guest,2016,246,"Oriol Paulo (dir.), Mario Casas, Ana Wagener",8.012836360773301,/title/tt4857264/
246,A Silent Voice: The Movie,2016,247,"Naoko Yamada (dir.), Miyu Irino, Saori Hayami",8.012503622007621,/title/tt5323662/
247,The Battle of Algiers,1966,248,"Gillo Pontecorvo (dir.), Brahim Hadjadj, Jean ...",8.012117378116962,/title/tt0058946/
248,Miracle in cell NO7,2019,249,"Mehmet Ada Öztekin (dir.), Aras Bulut Iynemli,...",8.011984697747106,/title/tt10431500/


In [127]:
# Lower case conversion
df['After_processing'] = df['crew'].apply(lambda item: " ".join(item.lower() for item in item.split()))
df


,movie,year,place,crew,rating,link,After_processing
0,The Shawshank Redemption,1994,1,"Frank Darabont (dir.), Tim Robbins, Morgan Fre...",9.223589219225481,/title/tt0111161/,"frank darabont (dir.), tim robbins, morgan fre..."
1,The Godfather,1972,2,"Francis Ford Coppola (dir.), Marlon Brando, Al...",9.147988016587115,/title/tt0068646/,"francis ford coppola (dir.), marlon brando, al..."
2,The Godfather: Part II,1974,3,"Francis Ford Coppola (dir.), Al Pacino, Robert...",8.980259662834687,/title/tt0071562/,"francis ford coppola (dir.), al pacino, robert..."
3,The Dark Knight,2008,4,"Christopher Nolan (dir.), Christian Bale, Heat...",8.974749960095577,/title/tt0468569/,"christopher nolan (dir.), christian bale, heat..."
4,12 Angry Men,1957,5,"Sidney Lumet (dir.), Henry Fonda, Lee J. Cobb",8.934368439017161,/title/tt0050083/,"sidney lumet (dir.), henry fonda, lee j. cobb"
...,...,...,...,...,...,...,...
245,The Invisible Guest,2016,246,"Oriol Paulo (dir.), Mario Casas, Ana Wagener",8.012836360773301,/title/tt4857264/,"oriol paulo (dir.), mario casas, ana wagener"
246,A Silent Voice: The Movie,2016,247,"Naoko Yamada (dir.), Miyu Irino, Saori Hayami",8.012503622007621,/title/tt5323662/,"naoko yamada (dir.), miyu irino, saori hayami"
247,The Battle of Algiers,1966,248,"Gillo Pontecorvo (dir.), Brahim Hadjadj, Jean ...",8.012117378116962,/title/tt0058946/,"gillo pontecorvo (dir.), brahim hadjadj, jean ..."
248,Miracle in cell NO7,2019,249,"Mehmet Ada Öztekin (dir.), Aras Bulut Iynemli,...",8.011984697747106,/title/tt10431500/,"mehmet ada öztekin (dir.), aras bulut iynemli,..."


In [128]:
# Removing punctuations
df['After_processing'] = df['After_processing'].str.replace('[^\w\s]','')
df

,movie,year,place,crew,rating,link,After_processing
0,The Shawshank Redemption,1994,1,"Frank Darabont (dir.), Tim Robbins, Morgan Fre...",9.223589219225481,/title/tt0111161/,frank darabont dir tim robbins morgan freeman
1,The Godfather,1972,2,"Francis Ford Coppola (dir.), Marlon Brando, Al...",9.147988016587115,/title/tt0068646/,francis ford coppola dir marlon brando al pacino
2,The Godfather: Part II,1974,3,"Francis Ford Coppola (dir.), Al Pacino, Robert...",8.980259662834687,/title/tt0071562/,francis ford coppola dir al pacino robert de niro
3,The Dark Knight,2008,4,"Christopher Nolan (dir.), Christian Bale, Heat...",8.974749960095577,/title/tt0468569/,christopher nolan dir christian bale heath ledger
4,12 Angry Men,1957,5,"Sidney Lumet (dir.), Henry Fonda, Lee J. Cobb",8.934368439017161,/title/tt0050083/,sidney lumet dir henry fonda lee j cobb
...,...,...,...,...,...,...,...
245,The Invisible Guest,2016,246,"Oriol Paulo (dir.), Mario Casas, Ana Wagener",8.012836360773301,/title/tt4857264/,oriol paulo dir mario casas ana wagener
246,A Silent Voice: The Movie,2016,247,"Naoko Yamada (dir.), Miyu Irino, Saori Hayami",8.012503622007621,/title/tt5323662/,naoko yamada dir miyu irino saori hayami
247,The Battle of Algiers,1966,248,"Gillo Pontecorvo (dir.), Brahim Hadjadj, Jean ...",8.012117378116962,/title/tt0058946/,gillo pontecorvo dir brahim hadjadj jean martin
248,Miracle in cell NO7,2019,249,"Mehmet Ada Öztekin (dir.), Aras Bulut Iynemli,...",8.011984697747106,/title/tt10431500/,mehmet ada öztekin dir aras bulut iynemli nisa...


In [129]:
# Removing numbers 
df['After_processing'] = df['After_processing'].apply(lambda item: ''.join([num for num in item if not num.isdigit()]))

In [130]:
# Removing special characters
import re
df['After_processing'] = df['After_processing'].apply(lambda item: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in item ))
df

,movie,year,place,crew,rating,link,After_processing
0,The Shawshank Redemption,1994,1,"Frank Darabont (dir.), Tim Robbins, Morgan Fre...",9.223589219225481,/title/tt0111161/,frank darabont dir tim robbins morgan freeman
1,The Godfather,1972,2,"Francis Ford Coppola (dir.), Marlon Brando, Al...",9.147988016587115,/title/tt0068646/,francis ford coppola dir marlon brando al pacino
2,The Godfather: Part II,1974,3,"Francis Ford Coppola (dir.), Al Pacino, Robert...",8.980259662834687,/title/tt0071562/,francis ford coppola dir al pacino robert de niro
3,The Dark Knight,2008,4,"Christopher Nolan (dir.), Christian Bale, Heat...",8.974749960095577,/title/tt0468569/,christopher nolan dir christian bale heath ledger
4,12 Angry Men,1957,5,"Sidney Lumet (dir.), Henry Fonda, Lee J. Cobb",8.934368439017161,/title/tt0050083/,sidney lumet dir henry fonda lee j cobb
...,...,...,...,...,...,...,...
245,The Invisible Guest,2016,246,"Oriol Paulo (dir.), Mario Casas, Ana Wagener",8.012836360773301,/title/tt4857264/,oriol paulo dir mario casas ana wagener
246,A Silent Voice: The Movie,2016,247,"Naoko Yamada (dir.), Miyu Irino, Saori Hayami",8.012503622007621,/title/tt5323662/,naoko yamada dir miyu irino saori hayami
247,The Battle of Algiers,1966,248,"Gillo Pontecorvo (dir.), Brahim Hadjadj, Jean ...",8.012117378116962,/title/tt0058946/,gillo pontecorvo dir brahim hadjadj jean martin
248,Miracle in cell NO7,2019,249,"Mehmet Ada Öztekin (dir.), Aras Bulut Iynemli,...",8.011984697747106,/title/tt10431500/,mehmet ada ztekin dir aras bulut iynemli nisa...


In [131]:
import nltk
nltk.download()


NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Package abc is already up-to-date!
       | Downloading package alpino to /root/nltk_data...
       |   Package alpino is already up-to-date!
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Package biocreative_ppi is already up-to-date!
       | Downloading package brown to /root/nltk_data...
       |   Package brown is already up-to-date!
       | Downloading package brown_tei to /root/nltk_data...
       |   Package brown_tei is already up-to-date!
       | Downloading package cess_cat to /root/nltk_data...
       |   Package cess_cat is already up-to-date!
       | Down

True

In [132]:
# Removing stop words
from nltk.corpus import stopwords
stop = stopwords.words('english')
df['After_processing'] = df['After_processing'].apply(lambda item: " ".join(item for item in item.split() if item not in stop))
df

,movie,year,place,crew,rating,link,After_processing
0,The Shawshank Redemption,1994,1,"Frank Darabont (dir.), Tim Robbins, Morgan Fre...",9.223589219225481,/title/tt0111161/,frank darabont dir tim robbins morgan freeman
1,The Godfather,1972,2,"Francis Ford Coppola (dir.), Marlon Brando, Al...",9.147988016587115,/title/tt0068646/,francis ford coppola dir marlon brando al pacino
2,The Godfather: Part II,1974,3,"Francis Ford Coppola (dir.), Al Pacino, Robert...",8.980259662834687,/title/tt0071562/,francis ford coppola dir al pacino robert de niro
3,The Dark Knight,2008,4,"Christopher Nolan (dir.), Christian Bale, Heat...",8.974749960095577,/title/tt0468569/,christopher nolan dir christian bale heath ledger
4,12 Angry Men,1957,5,"Sidney Lumet (dir.), Henry Fonda, Lee J. Cobb",8.934368439017161,/title/tt0050083/,sidney lumet dir henry fonda lee j cobb
...,...,...,...,...,...,...,...
245,The Invisible Guest,2016,246,"Oriol Paulo (dir.), Mario Casas, Ana Wagener",8.012836360773301,/title/tt4857264/,oriol paulo dir mario casas ana wagener
246,A Silent Voice: The Movie,2016,247,"Naoko Yamada (dir.), Miyu Irino, Saori Hayami",8.012503622007621,/title/tt5323662/,naoko yamada dir miyu irino saori hayami
247,The Battle of Algiers,1966,248,"Gillo Pontecorvo (dir.), Brahim Hadjadj, Jean ...",8.012117378116962,/title/tt0058946/,gillo pontecorvo dir brahim hadjadj jean martin
248,Miracle in cell NO7,2019,249,"Mehmet Ada Öztekin (dir.), Aras Bulut Iynemli,...",8.011984697747106,/title/tt10431500/,mehmet ada ztekin dir aras bulut iynemli nisa ...


In [133]:
#Stemming
from nltk.stem import PorterStemmer
st = PorterStemmer()
df['After_processing'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
df

,movie,year,place,crew,rating,link,After_processing
0,The Shawshank Redemption,1994,1,"Frank Darabont (dir.), Tim Robbins, Morgan Fre...",9.223589219225481,/title/tt0111161/,frank darabont dir tim robbins morgan freeman
1,The Godfather,1972,2,"Francis Ford Coppola (dir.), Marlon Brando, Al...",9.147988016587115,/title/tt0068646/,francis ford coppola dir marlon brando al pacino
2,The Godfather: Part II,1974,3,"Francis Ford Coppola (dir.), Al Pacino, Robert...",8.980259662834687,/title/tt0071562/,francis ford coppola dir al pacino robert de niro
3,The Dark Knight,2008,4,"Christopher Nolan (dir.), Christian Bale, Heat...",8.974749960095577,/title/tt0468569/,christopher nolan dir christian bale heath ledger
4,12 Angry Men,1957,5,"Sidney Lumet (dir.), Henry Fonda, Lee J. Cobb",8.934368439017161,/title/tt0050083/,sidney lumet dir henry fonda lee j cobb
...,...,...,...,...,...,...,...
245,The Invisible Guest,2016,246,"Oriol Paulo (dir.), Mario Casas, Ana Wagener",8.012836360773301,/title/tt4857264/,oriol paulo dir mario casas ana wagener
246,A Silent Voice: The Movie,2016,247,"Naoko Yamada (dir.), Miyu Irino, Saori Hayami",8.012503622007621,/title/tt5323662/,naoko yamada dir miyu irino saori hayami
247,The Battle of Algiers,1966,248,"Gillo Pontecorvo (dir.), Brahim Hadjadj, Jean ...",8.012117378116962,/title/tt0058946/,gillo pontecorvo dir brahim hadjadj jean martin
248,Miracle in cell NO7,2019,249,"Mehmet Ada Öztekin (dir.), Aras Bulut Iynemli,...",8.011984697747106,/title/tt10431500/,mehmet ada ztekin dir aras bulut iynemli nisa ...


In [ ]:
#Spelling corrections
from textblob import TextBlob
df['After_processing'].apply(lambda x: str(TextBlob(x).correct()))
df

In [134]:
# Lemmatization
from textblob import Word
import nltk
nltk.download('wordnet')

df['After_processing'] = df['After_processing'].apply(lambda item: " ".join([Word(word).lemmatize() for word in item.split()]))
df

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,movie,year,place,crew,rating,link,After_processing
0,The Shawshank Redemption,1994,1,"Frank Darabont (dir.), Tim Robbins, Morgan Fre...",9.223589219225481,/title/tt0111161/,frank darabont dir tim robbins morgan freeman
1,The Godfather,1972,2,"Francis Ford Coppola (dir.), Marlon Brando, Al...",9.147988016587115,/title/tt0068646/,francis ford coppola dir marlon brando al pacino
2,The Godfather: Part II,1974,3,"Francis Ford Coppola (dir.), Al Pacino, Robert...",8.980259662834687,/title/tt0071562/,francis ford coppola dir al pacino robert de niro
3,The Dark Knight,2008,4,"Christopher Nolan (dir.), Christian Bale, Heat...",8.974749960095577,/title/tt0468569/,christopher nolan dir christian bale heath ledger
4,12 Angry Men,1957,5,"Sidney Lumet (dir.), Henry Fonda, Lee J. Cobb",8.934368439017161,/title/tt0050083/,sidney lumet dir henry fonda lee j cobb
...,...,...,...,...,...,...,...
245,The Invisible Guest,2016,246,"Oriol Paulo (dir.), Mario Casas, Ana Wagener",8.012836360773301,/title/tt4857264/,oriol paulo dir mario casas ana wagener
246,A Silent Voice: The Movie,2016,247,"Naoko Yamada (dir.), Miyu Irino, Saori Hayami",8.012503622007621,/title/tt5323662/,naoko yamada dir miyu irino saori hayami
247,The Battle of Algiers,1966,248,"Gillo Pontecorvo (dir.), Brahim Hadjadj, Jean ...",8.012117378116962,/title/tt0058946/,gillo pontecorvo dir brahim hadjadj jean martin
248,Miracle in cell NO7,2019,249,"Mehmet Ada Öztekin (dir.), Aras Bulut Iynemli,...",8.011984697747106,/title/tt10431500/,mehmet ada ztekin dir ara bulut iynemli nisa s...


In [137]:
#Parts of Speech Tagging and Features
from nltk.tokenize import word_tokenize
pos = []
for sentence in df['After_processing']:
  text = word_tokenize(sentence)
  pos.append(nltk.pos_tag(text))
pos

[[('frank', 'JJ'),
  ('darabont', 'NN'),
  ('dir', 'NN'),
  ('tim', 'NN'),
  ('robbins', 'VBZ'),
  ('morgan', 'JJ'),
  ('freeman', 'NN')],
 [('francis', 'NN'),
  ('ford', 'NN'),
  ('coppola', 'NN'),
  ('dir', 'NN'),
  ('marlon', 'NN'),
  ('brando', 'NN'),
  ('al', 'NN'),
  ('pacino', 'NN')],
 [('francis', 'NN'),
  ('ford', 'NN'),
  ('coppola', 'VBZ'),
  ('dir', 'JJ'),
  ('al', 'NN'),
  ('pacino', 'NN'),
  ('robert', 'NN'),
  ('de', 'IN'),
  ('niro', 'FW')],
 [('christopher', 'NN'),
  ('nolan', 'NN'),
  ('dir', 'VBZ'),
  ('christian', 'JJ'),
  ('bale', 'JJ'),
  ('heath', 'NN'),
  ('ledger', 'NN')],
 [('sidney', 'NN'),
  ('lumet', 'NN'),
  ('dir', 'NN'),
  ('henry', 'NN'),
  ('fonda', 'NN'),
  ('lee', 'VBZ'),
  ('j', 'NNS'),
  ('cobb', 'NN')],
 [('steven', 'RB'),
  ('spielberg', 'JJ'),
  ('dir', 'NN'),
  ('liam', 'NN'),
  ('neeson', 'NN'),
  ('ralph', 'NN'),
  ('fiennes', 'NNS')],
 [('peter', 'NN'),
  ('jackson', 'NN'),
  ('dir', 'NN'),
  ('elijah', 'NN'),
  ('wood', 'NN'),
  ('viggo', '

In [138]:
Adjective = []
Adverb = []
CordinatingConjunction = []
SubordinatingConjuction = []
Interjection = []
Noun = []
Verb = []
PersonalPronoun = []
predeterminer = []
Determiner = []

In [139]:
for value in pos:
  AdjectiveCount = 0
  AdverbCount = 0
  CordinatingConjunctionCount = 0
  SubordinatingConjuctionCount = 0
  InterjectionCount = 0
  NounCount = 0
  VerbCount = 0
  PersonalPronounCount = 0
  predeterminerCount = 0
  DeterminerCount = 0
  for word,tag in value:
    if tag == 'JJ':
      AdjectiveCount = AdjectiveCount + 1
    elif tag == 'RB':
      AdverbCount = AdverbCount + 1
    elif tag == 'CC':
      CordinatingConjunctionCount = CordinatingConjunctionCount + 1
    elif tag == 'UH':
      InterjectionCount = InterjectionCount + 1
    elif tag == 'NN':
      NounCount = NounCount + 1
    elif tag == 'VR':
      VerbCount = VerbCount + 1
    elif tag == 'PRP':
      PersonalPronounCount = PersonalPronounCount + 1
    elif tag == 'PDT':
      predeterminerCount = predeterminerCount + 1
    elif tag == 'DT':
      DeterminerCount = DeterminerCount + 1
    elif tag == 'IN':
      SubordinatingConjuctionCount = SubordinatingConjuctionCount + 1
  Adjective.append(AdjectiveCount)
  Adverb.append(AdverbCount)
  CordinatingConjunction.append(CordinatingConjunctionCount)
  Interjection.append(InterjectionCount)
  Noun.append(NounCount)
  Verb.append(VerbCount)
  PersonalPronoun.append(PersonalPronounCount)
  predeterminer.append(predeterminerCount)
  Determiner.append(DeterminerCount)
  SubordinatingConjuction.append(SubordinatingConjuctionCount)

In [140]:
df['Number of Adjectives'] = Adjective
df['Number of Adverbs'] = Adverb
df['Number of Cordinating Conjunctions'] = CordinatingConjunction
df['Number of Interjections'] = Interjection
df['Number of Nouns'] = Noun
df['Number of Verbs'] = Verb
df['Number of Personal Pronouns'] = PersonalPronoun
df['Number of Predeterminers'] = predeterminer
df['Number of Determiners'] = Determiner
df['Number of Subordinating Conjuctions'] = SubordinatingConjuction
df

,movie,year,place,crew,rating,link,After_processing,Number of Adjectives,Number of Adverbs,Number of Cordinating Conjunctions,Number of Interjections,Number of Nouns,Number of Verbs,Number of Personal Pronouns,Number of Predeterminers,Number of Determiners,Number of Subordinating Conjuctions
0,The Shawshank Redemption,1994,1,"Frank Darabont (dir.), Tim Robbins, Morgan Fre...",9.223589219225481,/title/tt0111161/,frank darabont dir tim robbins morgan freeman,2,0,0,0,4,0,0,0,0,0
1,The Godfather,1972,2,"Francis Ford Coppola (dir.), Marlon Brando, Al...",9.147988016587115,/title/tt0068646/,francis ford coppola dir marlon brando al pacino,0,0,0,0,8,0,0,0,0,0
2,The Godfather: Part II,1974,3,"Francis Ford Coppola (dir.), Al Pacino, Robert...",8.980259662834687,/title/tt0071562/,francis ford coppola dir al pacino robert de niro,1,0,0,0,5,0,0,0,0,1
3,The Dark Knight,2008,4,"Christopher Nolan (dir.), Christian Bale, Heat...",8.974749960095577,/title/tt0468569/,christopher nolan dir christian bale heath ledger,2,0,0,0,4,0,0,0,0,0
4,12 Angry Men,1957,5,"Sidney Lumet (dir.), Henry Fonda, Lee J. Cobb",8.934368439017161,/title/tt0050083/,sidney lumet dir henry fonda lee j cobb,0,0,0,0,6,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,The Invisible Guest,2016,246,"Oriol Paulo (dir.), Mario Casas, Ana Wagener",8.012836360773301,/title/tt4857264/,oriol paulo dir mario casas ana wagener,1,0,0,0,6,0,0,0,0,0
246,A Silent Voice: The Movie,2016,247,"Naoko Yamada (dir.), Miyu Irino, Saori Hayami",8.012503622007621,/title/tt5323662/,naoko yamada dir miyu irino saori hayami,1,1,0,0,3,0,1,0,0,0
247,The Battle of Algiers,1966,248,"Gillo Pontecorvo (dir.), Brahim Hadjadj, Jean ...",8.012117378116962,/title/tt0058946/,gillo pontecorvo dir brahim hadjadj jean martin,1,0,0,0,6,0,0,0,0,0
248,Miracle in cell NO7,2019,249,"Mehmet Ada Öztekin (dir.), Aras Bulut Iynemli,...",8.011984697747106,/title/tt10431500/,mehmet ada ztekin dir ara bulut iynemli nisa s...,1,0,0,0,8,0,0,0,0,0


Linguistic Features

In [142]:
RightBranchingNodes = []
nlp = spacy.load("en_core_web_sm")
for sentence in df['After_processing']:
  doc = nlp(sentence)
  try:
    RightBranchingNodes.append(doc[0].n_rights)
  except:
    RightBranchingNodes.append('No')
df['Number of Right Branching Nodes'] = RightBranchingNodes


In [144]:
#Sentence Length 

df['Sentenece Length'] = df['After_processing'].apply(lambda x: len(x))
df

,movie,year,place,crew,rating,link,After_processing,Number of Adjectives,Number of Adverbs,Number of Cordinating Conjunctions,Number of Interjections,Number of Nouns,Number of Verbs,Number of Personal Pronouns,Number of Predeterminers,Number of Determiners,Number of Subordinating Conjuctions,Number of Right Branching Nodes,Sentenece Length
0,The Shawshank Redemption,1994,1,"Frank Darabont (dir.), Tim Robbins, Morgan Fre...",9.223589219225481,/title/tt0111161/,frank darabont dir tim robbins morgan freeman,2,0,0,0,4,0,0,0,0,0,0,45
1,The Godfather,1972,2,"Francis Ford Coppola (dir.), Marlon Brando, Al...",9.147988016587115,/title/tt0068646/,francis ford coppola dir marlon brando al pacino,0,0,0,0,8,0,0,0,0,0,0,48
2,The Godfather: Part II,1974,3,"Francis Ford Coppola (dir.), Al Pacino, Robert...",8.980259662834687,/title/tt0071562/,francis ford coppola dir al pacino robert de niro,1,0,0,0,5,0,0,0,0,1,0,49
3,The Dark Knight,2008,4,"Christopher Nolan (dir.), Christian Bale, Heat...",8.974749960095577,/title/tt0468569/,christopher nolan dir christian bale heath ledger,2,0,0,0,4,0,0,0,0,0,0,49
4,12 Angry Men,1957,5,"Sidney Lumet (dir.), Henry Fonda, Lee J. Cobb",8.934368439017161,/title/tt0050083/,sidney lumet dir henry fonda lee j cobb,0,0,0,0,6,0,0,0,0,0,0,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,The Invisible Guest,2016,246,"Oriol Paulo (dir.), Mario Casas, Ana Wagener",8.012836360773301,/title/tt4857264/,oriol paulo dir mario casas ana wagener,1,0,0,0,6,0,0,0,0,0,0,39
246,A Silent Voice: The Movie,2016,247,"Naoko Yamada (dir.), Miyu Irino, Saori Hayami",8.012503622007621,/title/tt5323662/,naoko yamada dir miyu irino saori hayami,1,1,0,0,3,0,1,0,0,0,0,40
247,The Battle of Algiers,1966,248,"Gillo Pontecorvo (dir.), Brahim Hadjadj, Jean ...",8.012117378116962,/title/tt0058946/,gillo pontecorvo dir brahim hadjadj jean martin,1,0,0,0,6,0,0,0,0,0,0,47
248,Miracle in cell NO7,2019,249,"Mehmet Ada Öztekin (dir.), Aras Bulut Iynemli,...",8.011984697747106,/title/tt10431500/,mehmet ada ztekin dir ara bulut iynemli nisa s...,1,0,0,0,8,0,0,0,0,0,0,60
